In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [35]:
columns = ["month_date_yyyymm","postal_code","zip_name","median_listing_price_mm","median_listing_price_yy","active_listing_count"
           "active_listing_count_mm","active_listing_count_yy","median_days_on_market","average_listing_price_mm","average_listing_price_yy",
           'total_listing_count',"total_listing_count_mm",'total_listing_count_yy',"pending_ratio","pending_ratio_mm","pending_ratio_yy","quality_flag","year-month"]

target = ["median_listing_price"]

In [36]:
# Load the data
file_path = Path('/Users/davidpruette/Desktop/UNC_Class/FinalProject/Resources/data_nona.csv')
data_nona = pd.read_csv(file_path)
data_nona.head()

,month_date_yyyymm,postal_code,zip_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag,year-month
0,202206,33572,"apollo beach, fl",731950.0,0.1665,0.5577,93.0,0.4091,0.9787,24.0,...,0.0181,0.2696,280.0,0.1382,0.3146,1.9785,-0.7336,-1.5534,1.0,2022-06-01
1,202206,28445,"holly ridge, nc",687450.0,0.1325,0.4761,77.0,0.3750,0.2833,33.0,...,0.0795,0.1772,317.0,0.0893,0.0063,3.1429,-1.0714,-1.0905,1.0,2022-06-01
2,202206,78634,"hutto, tx",472500.0,0.0071,0.2459,134.0,0.6962,0.8356,16.0,...,-0.0249,0.2655,182.0,0.4677,0.3000,0.3358,-0.2465,-0.5820,1.0,2022-06-01
3,202206,20872,"damascus, md",731995.0,0.0028,0.2642,16.0,-0.0588,-0.1111,53.0,...,-0.0137,0.1910,36.0,0.0909,-0.2000,1.2500,0.3676,-0.1944,1.0,2022-06-01
4,202206,87121,"albuquerque, nm",271000.0,0.0131,0.2047,35.0,-0.1026,0.2069,36.0,...,0.0424,0.2202,159.0,-0.0305,-0.0185,3.5429,0.3634,-1.0089,1.0,2022-06-01


In [37]:
data_nona = data_nona.drop(columns=["zip_name"])
data_nona = data_nona.drop(columns=['year-month'])
data_nona.head()

,month_date_yyyymm,postal_code,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,202206,33572,731950.0,0.1665,0.5577,93.0,0.4091,0.9787,24.0,-0.0204,...,861186.0,0.0181,0.2696,280.0,0.1382,0.3146,1.9785,-0.7336,-1.5534,1.0
1,202206,28445,687450.0,0.1325,0.4761,77.0,0.3750,0.2833,33.0,-0.1538,...,752644.0,0.0795,0.1772,317.0,0.0893,0.0063,3.1429,-1.0714,-1.0905,1.0
2,202206,78634,472500.0,0.0071,0.2459,134.0,0.6962,0.8356,16.0,0.0000,...,529484.0,-0.0249,0.2655,182.0,0.4677,0.3000,0.3358,-0.2465,-0.5820,1.0
3,202206,20872,731995.0,0.0028,0.2642,16.0,-0.0588,-0.1111,53.0,0.6935,...,839165.0,-0.0137,0.1910,36.0,0.0909,-0.2000,1.2500,0.3676,-0.1944,1.0
4,202206,87121,271000.0,0.0131,0.2047,35.0,-0.1026,0.2069,36.0,0.2456,...,308163.0,0.0424,0.2202,159.0,-0.0305,-0.0185,3.5429,0.3634,-1.0089,1.0


In [38]:
X = pd.get_dummies(data_nona.drop(columns="median_listing_price"))

y = data_nona["median_listing_price"]

In [39]:
X.describe()

,month_date_yyyymm,postal_code,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
count,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,...,9.932700e+04,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000,99327.000000
mean,201917.777080,57111.627594,0.007933,0.088851,143.596545,0.008215,-0.044435,51.680449,0.024267,-0.028579,...,5.241404e+05,0.008462,0.090531,239.164668,0.003286,-0.008881,1.056436,0.005931,0.213783,0.217705
std,150.088201,27961.052961,0.055088,0.152940,150.422068,0.161327,0.428071,24.091288,0.235440,0.360604,...,5.035048e+05,0.125404,0.202957,191.016174,0.113625,0.298722,1.079800,0.455752,0.980579,0.412688
min,201707.000000,1028.000000,-0.586400,-0.665300,3.000000,-0.848000,-0.971200,2.000000,-0.916700,-0.950000,...,3.922700e+04,-0.971100,-0.928700,8.000000,-0.707100,-0.824500,0.000000,-11.857100,-24.974400,0.000000
25%,201806.000000,32174.000000,-0.016400,0.001900,55.000000,-0.076900,-0.292700,36.000000,-0.107800,-0.230800,...,2.958290e+05,-0.017400,-0.002200,118.000000,-0.058100,-0.184400,0.424000,-0.091500,-0.105700,0.000000
50%,201904.000000,55328.000000,0.002400,0.065300,100.000000,0.000000,-0.073700,47.000000,0.020800,-0.064100,...,4.046320e+05,0.004300,0.063800,186.000000,-0.003800,-0.045500,0.722200,-0.001600,0.043900,0.000000
75%,202010.000000,85032.000000,0.028000,0.147100,178.000000,0.074600,0.133300,63.000000,0.144400,0.113600,...,5.765560e+05,0.028400,0.150400,299.000000,0.055350,0.109500,1.340300,0.092000,0.357200,0.000000
max,202206.000000,99353.000000,1.011400,4.528400,2658.000000,3.000000,22.000000,543.000000,4.250000,13.666700,...,1.943063e+07,32.093700,31.623700,2878.000000,2.935900,8.592600,89.666700,56.916700,87.809500,1.000000


In [41]:
y.value_counts()

299900.0    398
249900.0    371
325000.0    371
349900.0    342
425000.0    321
           ... 
254959.0      1
318131.0      1
289123.0      1
411083.0      1
255975.0      1
Name: median_listing_price, Length: 27986, dtype: int64

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [11]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators =100, random_state=1)
brfc_model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brfc_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7877672625306695

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 1560, 15558]])

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [16]:
# List the features sorted in descending order by feature importance
priority_features = pd.Series(data=brfc_model.feature_importances_,index=X.columns)
priority_features.sort_values(ascending=False,inplace=True)
print(priority_features)

total_rec_prncp             0.073767
total_rec_int               0.063903
total_pymnt_inv             0.060733
total_pymnt                 0.058112
last_pymnt_amnt             0.049518
                              ...   
acc_now_delinq              0.000000
delinq_amnt                 0.000000
chargeoff_within_12_mths    0.000000
hardship_flag_N             0.000000
debt_settlement_flag_N      0.000000
Length: 95, dtype: float64


### Easy Ensemble AdaBoost Classifier

In [17]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec_model = EasyEnsembleClassifier(n_estimators =100, random_state=1)
eec_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [18]:
# Calculated the balanced accuracy score
y_pred = eec_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.925427358175101

In [19]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  979, 16139]])

In [20]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205

